In [8]:
N = 3000
y_true = np.random.randint(0, 2, N)
y_pred = np.random.randn(N)

exp_pred = np.exp(y_pred)

In [9]:
sigmoid_pairwise_loss(y_true, exp_pred, 1)

-4088677.6148932595

In [10]:
sigmoid_pairwise_loss(y_true, exp_pred, 12)

-4088677.6148932157

In [3]:
%%timeit
sigmoid_pairwise_diff_hess(y_true, exp_pred, 12)

1.18 s ± 26.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
%%time
sigmoid_pairwise_diff_hess(y_true, exp_pred)

NameError: name 'exp_pred' is not defined

In [1]:
def get_pairs(y_true, y_pred):
    pairs = []
    for i in range(len(y_true)):
        for j in range(i + 1):
            pairs.append([(i, j), (y_true[i], y_true[j]), (y_pred[i], y_pred[j])])
    return pairs

In [7]:
import numpy as np
y_true = [0 for i in range(10000)]
y_pred = [np.log(1) for i in range(10000)]

In [3]:
pairs = get_pairs(y_true, y_pred)

In [4]:
from collections import defaultdict

In [5]:
def grad(pairs):
    grad_ = defaultdict(lambda : 0)
    hess_ = defaultdict(lambda : 0)
    for p in pairs:
        (i, j), (y_true_i, y_true_j), (y_pred_i, y_pred_j) = p
        if(y_true_i > y_true_j):
            P_hat = 1.0
        elif y_true_j == y_true_i:
            P_hat = 0.5
        elif y_true_i < y_true_j:
            P_hat = 0.0
        grad_i = ((P_hat - 1.)*np.exp(y_pred_i) + P_hat*np.exp(y_pred_j)) / (np.exp(y_pred_i) + np.exp(y_pred_j))
        grad_j = -grad_i

        hess_i = -(np.exp(y_pred_i + y_pred_j)) / (np.exp(y_pred_i) + np.exp(y_pred_j))**2
        hess_j = hess_i

        grad_[i] += grad_i
        grad_[j] += grad_j
        
        hess_[i] += hess_i
        hess_[j] += hess_j
    return grad_.values(), hess_.values()

In [6]:
grad(pairs=pairs)

(dict_values([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

In [7]:
tmp[0].values()

NameError: name 'tmp' is not defined

In [1]:
from rocauc_pairwise.sigmoid_pairwise_cpu import sigmoid_pairwise_diff_hess
import numpy as np

In [2]:
y_true = [0 for i in range(10000)]
y_pred = [np.log(1) for i in range(10000)]

In [4]:
sigmoid_pairwise_diff_hess(np.array(y_true), np.exp(y_pred), 12)

(array([0., 0., 0., ..., 0., 0., 0.]),
 array([-2500.25, -2500.25, -2500.25, ..., -2500.25, -2500.25, -2500.25]))

In [3]:
%%timeit
_, _ = sigmoid_pairwise_diff_hess(np.array(y_true), np.exp(y_pred), 12)

1.22 s ± 47.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
%%timeit
_, _ = sigmoid_pairwise_diff_hess(np.array(y_true), np.exp(y_pred), 12)

1.18 s ± 46.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
%%timeit
_, _ = sigmoid_pairwise_diff_hess(np.array(y_true), np.exp(y_pred), 12)

1.03 s ± 53.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
np.sort([(15*i) %16 for i in range(16])

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

In [5]:
def f(i, size):
    if i % 2 == 1:
        i = size - i//2 - 1
    else:
        i = i // 2
    return i

In [6]:
[f(i, 3) for i in range(3)]

[0, 2, 1]